<a href="https://colab.research.google.com/github/Lavender0916/gpt-ai-assistant/blob/main/%E5%B0%88%E9%A1%8C1022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install dash dash-bootstrap-components pandas yfinance plotly

In [6]:
import dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# 初始化 Dash 应用，使用 Bootstrap 主题
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# 定义 ETF 代码和名称的字典
etf_dict = {
    "0050": "元大台灣50",
    "0051": "元大中型100",
    "0052": "富邦科技",
    "0053": "元大電子",
    "0055": "元大MSCI金融",
    "0056": "元大高股息",
    "0057": "富邦摩台",
    "006201": "元大富櫃50",
    "006203": "元大MSCI台灣",
    "006204": "永豐臺灣加權",
    "006208": "富邦台50",
    "00631L": "元大台灣50正2",
    "00632R": "元大台灣50反1",
    "00663L": "國泰臺灣加權正2",
    "00664R": "國泰臺灣加權反1",
    "00675L": "富邦臺灣加權正2",
    "00676R": "富邦臺灣加權反1",
    "00685L": "群益臺灣加權正2",
    "00686R": "群益臺灣加權反1",
    "00690": "兆豐藍籌30",
    "00692": "富邦公司治理",
    "00701": "國泰股利精選30",
    "00713": "元大台灣高息低波",
    "00728": "第一金工業30",
    "00730": "富邦臺灣優質高息",
    "00731": "復華富時高息低波",
    "00733": "富邦臺灣中小",
    "00735": "國泰臺韓科技",
    "00850": "元大臺灣ESG永續",
    "00878": "國泰永續高股息",
    "00881": "國泰台灣5G+",
    "00888": "永豐台灣ESG",
    "00891": "中信關鍵半導體",
    "00892": "富邦台灣半導體",
    "00894": "中信小資高價30",
    "00896": "中信綠能及電動車",
    "00900": "富邦特選高股息30",
    "00901": "永豐智能車供應鏈",
    "00904": "新光臺灣半導體30",
    "00905": "FT臺灣Smart",
    "00907": "永豐優息存股",
    "00912": "中信臺灣智慧50",
    "00913": "兆豐台灣晶圓製造",
    "00915": "凱基優選高股息30",
    "00918": "大華優利高填息30",
    "00919": "群益台灣精選高息",
    "00921": "兆豐龍頭等權重",
    "00922": "國泰台灣領袖50",
    "00923": "群益台ESG低碳50",
    "00927": "群益半導體收益",
    "00928": "中信上櫃ESG30",
    "00929": "復華台灣科技優息",
    "00930": "永豐ESG低碳高息",
    "00932": "兆豐永續高息等權",
    "00934": "中信成長高股息",
    "00935": "野村臺灣新科技50",
    "00936": "台新永續高息中小",
    "00939": "統一台灣高息動能",
    "00940": "元大台灣價值高息",
    "00943": "兆豐電子高息等權",
    "00944": "野村趨勢動能高息",
    "00946": "群益科技高息成長",
    "00947": "台新臺灣IC設計",
    "00952": "凱基台灣A150"
}

# 应用布局
app.layout = dbc.Container([
    html.H1("台湾 ETF 资讯与图表", className='text-center mt-4 mb-4'),

    # 下拉选单
    dcc.Dropdown(
        id='etf-dropdown',
        options=[{'label': f"{symbol} - {name}", 'value': symbol} for symbol, name in etf_dict.items()],
        value='00937B',  # 默认选择
        className='mb-4',
        style={'width': '50%', 'margin': '0 auto'}  # 调整宽度并居中
    ),

    # 时间范围滑块
    dcc.RangeSlider(
        id='date-slider',
        min=0,
        max=23,
        step=1,
        marks={i: f'{2023 + i // 12}-{i % 12 + 1:02d}' for i in range(24)},
        value=[0, 11],
        tooltip={"placement": "bottom", "always_visible": True},
        className='mb-4'
    ),

    # 三个区块布局
    dbc.Row([
        # 左上区块：K 线图
        dbc.Col(dcc.Graph(id='kline-chart'), width=6),
        # 右上区块：折线图 + 成交量长条图
        dbc.Col(dcc.Graph(id='combined-chart'), width=6)
    ]),

    # 下方区块：数据表格
    dbc.Row([
        dbc.Col(dash_table.DataTable(
            id='data-table',
            style_table={'overflowX': 'auto'},
            style_data_conditional=[
                {
                    'if': {
                        'filter_query': '{Change} > 0',
                        'column_id': 'Price'
                    },
                    'color': 'red'
                },
                {
                    'if': {
                        'filter_query': '{Change} < 0',
                        'column_id': 'Price'
                    },
                    'color': 'green'
                },
                {
                    'if': {
                        'filter_query': '{Change} > 0',
                        'column_id': 'Change'
                    },
                    'color': 'red'
                },
                {
                    'if': {
                        'filter_query': '{Change} < 0',
                        'column_id': 'Change'
                    },
                    'color': 'green'
                },
                {
                    'if': {
                        'filter_query': '{Change} > 0',
                        'column_id': 'Change %'
                    },
                    'color': 'red'
                },
                {
                    'if': {
                        'filter_query': '{Change} < 0',
                        'column_id': 'Change %'
                    },
                    'color': 'green'
                }
            ]
        ), width=12)
    ])
])

# 更新图表和数据表格
@app.callback(
    [Output('kline-chart', 'figure'),
     Output('combined-chart', 'figure'),
     Output('data-table', 'data')],
    [Input('etf-dropdown', 'value'),
     Input('date-slider', 'value')]
)
def update_charts(selected_etf, date_range):
    # 获取 ETF 数据
    df = yf.download(selected_etf + '.TW', start='2023-01-01', end='2024-12-31')
    df_two = yf.download(selected_etf + '.TWO', start='2023-01-01', end='2024-12-31')

    # 将 .TWO 的数据合并到 .TW 的数据中
    df = pd.concat([df, df_two], keys=['.TW', '.TWO']).reset_index(level=0).rename(columns={'level_0': 'Source'})

    if df.empty:
        return go.Figure(), go.Figure(), []

    # 添加日期索引
    df['Date'] = df.index

    # 将日期转换为月份索引
    df['Month'] = df['Date'].dt.month - 1 + (df['Date'].dt.year - 2023) * 12

    # 根据滑块的值过滤数据
    filtered_df = df[(df['Month'] >= date_range[0]) & (df['Month'] <= date_range[1])]

    if filtered_df.empty:
        return go.Figure(), go.Figure(), []

    # 计算股价、涨跌和涨跌幅
    filtered_df = filtered_df.reset_index()
    filtered_df['Price'] = filtered_df['Close']
    filtered_df['Change'] = filtered_df['Close'] - filtered_df['Open']
    filtered_df['Change %'] = (filtered_df['Change'] / filtered_df['Open']) * 100

    # K 线图
    kline_fig = go.Figure(data=[go.Candlestick(
        x=filtered_df['Date'],
        open=filtered_df['Open'],
        high=filtered_df['High'],
        low=filtered_df['Low'],
        close=filtered_df['Close']
    )])
    kline_fig.update_layout(title=f'{selected_etf} K 线图', xaxis_title='日期', yaxis_title='价格')

    # 折线图 + 成交量长条图
    combined_fig = go.Figure()
    combined_fig.add_trace(go.Scatter(x=filtered_df['Date'], y=filtered_df['Open'], mode='lines', name='开盘价'))
    combined_fig.add_trace(go.Scatter(x=filtered_df['Date'], y=filtered_df['Close'], mode='lines', name='收盘价'))
    combined_fig.add_trace(go.Scatter(x=filtered_df['Date'], y=filtered_df['High'], mode='lines', name='最高价'))
    combined_fig.add_trace(go.Scatter(x=filtered_df['Date'], y=filtered_df['Low'], mode='lines', name='最低价'))
    combined_fig.add_trace(go.Bar(x=filtered_df['Date'], y=filtered_df['Volume'], name='成交量', yaxis='y2'))
    combined_fig.update_layout(
        title=f'{selected_etf} 价格与成交量',
        xaxis_title='日期',
        yaxis_title='价格',
        yaxis2=dict(title='成交量', overlaying='y', side='right'),
        legend=dict(x=0, y=1, bgcolor='rgba(255,255,255,0)')
    )

    # 数据表格，格式化价格到小数点后两位
    table_data = filtered_df[['Date', 'Price', 'Change', 'Change %', 'Open', 'Close', 'High', 'Low', 'Volume']].copy()
    table_data['Date'] = table_data['Date'].dt.strftime('%Y-%m-%d')  # 格式化日期
    table_data['Price'] = table_data['Price'].apply(lambda x: f"{x:.2f}")
    table_data['Change'] = table_data['Change'].apply(lambda x: f"{x:.2f}")
    table_data['Change %'] = table_data['Change %'].apply(lambda x: f"{x:.2f}")
    table_data['Open'] = table_data['Open'].apply(lambda x: f"{x:.2f}")  # 格式化开盘价
    table_data['Close'] = table_data['Close'].apply(lambda x: f"{x:.2f}")  # 格式化收盘价
    table_data['High'] = table_data['High'].apply(lambda x: f"{x:.2f}")  # 格式化最高价
    table_data['Low'] = table_data['Low'].apply(lambda x: f"{x:.2f}")  # 格式化最低价
    table_data['Volume'] = table_data['Volume'].apply(lambda x: f"{int(x):,}")  # 格式化成交量为整数

    return kline_fig, combined_fig, table_data.to_dict('records')

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>